# 关联分析的优化：FP-growth算法

### 1 概述

FP-growth是一种高效发现频繁集的方法。FP代表频繁模式（Frequent Pattern）。

FP-growth算法只需要对数据库进行两次扫描，而Apriori算法对于每个潜在的频繁项集都会扫描数据集判定给定模式是否频繁，因此FP-growth算法的速度要比Apriori算法快。

它发现频繁项集的基本过程如下：
1. 构建FP树
2. 从FP树中挖掘频繁项集

* 优点：一般要快于Apriori。
* 缺点：实现比较困难，在某些数据集上性能会下降。
* 适用数据类型：标称型数据。

FP-growth算法首先构建FP树，然后利用它来挖掘频繁项集。为构建FP树，需要对原始数据集扫描两遍。第一遍对所有元素项的出现次数进行计数。由Apriori原理，如果某元素是不频繁的，那么包含该元素的超集也是不频繁的，所以就不需要考虑这些超集。数据库的第一遍扫描用来统计出现的频率，而第二遍扫描中只考虑那些频繁元素。

### 2 FP树的构建

1. 收集数据：使用任意方法。
2. 准备数据：由于存储的是集合，所以需要离散数据。如果要处理连续数据，需要将它们量化为离散值。
3. 分析数据：使用任意方法。
4. 训练算法：构建一个FP树，并对树进行挖掘。
5. 测试算法：没有测试过程。
6. 使用算法：可用于识别经常出现的元素项，从而用于制定决策、推荐元素或进行预测等应用中。

In [1]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {} 
    
    def inc(self, numOccur):
        self.count += numOccur
        
    def disp(self, ind=1):
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [2]:
rootNode = treeNode('pyramid', 9, None)

In [3]:
rootNode.children['eye'] = treeNode('eye', 13, None)

In [4]:
rootNode.disp()

   pyramid   9
     eye   13


In [5]:
def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    headerTable = {}
    #go over dataSet twice
    for trans in dataSet:#first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    
    # 移除不满足最小支持度的元素项
    for k in list(headerTable.keys()):
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print 'freqItemSet: ',freqItemSet
    
    # 如果没有元素项满足要求则退出
    if len(freqItemSet) == 0: return None, None
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = {}
        
        # 根据全局频率对每个事务中的元素进行排序
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            
            # 使用排序后的频率项集对树进行填充
            updateTree(orderedItems, retTree, headerTable, count)#populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table

def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:
        # 对剩下的元素项迭代调用updateTree()
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)
        
def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [6]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [7]:
simpDat = loadSimpDat()
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [8]:
initSet = createInitSet(simpDat)
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [9]:
myFPtree, myHeaderTab = createTree(initSet, 3)
myFPtree.disp()

   Null Set   1
     z   5
       r   1
       x   3
         s   2
           y   2
             t   2
         y   1
           t   1
             r   1
     x   1
       s   1
         r   1


### 3 从一棵FP 树中挖掘频繁项集

从FP树中抽取频繁项集的三个基本步骤如下：

1. 从FP树中获得条件模式基；
2. 利用条件模式基，构建一个条件FP树；
3. 迭代重复步骤1步骤2，直到树包含一个元素项为止。

接下来重点关注第1步，即寻找条件模式基的过程。之后，为每一个条件模式基创建对应的
条件FP树。最后需要构造少许代码来封装上述两个函数，并从FP树中获得频繁项集。

In [10]:
def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)
    
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [11]:
findPrefixPath('x', myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [12]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1][0])]#(sort header table)
    for basePat in bigL:  #start from bottom of header table
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        #print 'finalFrequent Item: ',newFreqSet    #append to set
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        #print 'condPattBases :',basePat, condPattBases
        #2. construct cond FP-tree from cond. pattern base
        myCondTree, myHead = createTree(condPattBases, minSup)
        #print 'head from conditional tree: ', myHead
        if myHead != None: #3. mine cond. FP-tree
            print('conditional tree for: ',newFreqSet)
            myCondTree.disp(1)
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [13]:
freqItems = []
mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)

conditional tree for:  {'s'}
   Null Set   1
     x   3
conditional tree for:  {'y'}
   Null Set   1
     z   3
       x   3
conditional tree for:  {'y', 'x'}
   Null Set   1
     z   3
conditional tree for:  {'t'}
   Null Set   1
     y   3
       x   3
         z   3
conditional tree for:  {'t', 'x'}
   Null Set   1
     y   3
conditional tree for:  {'t', 'z'}
   Null Set   1
     y   3
       x   3
conditional tree for:  {'t', 'z', 'x'}
   Null Set   1
     y   3
conditional tree for:  {'x'}
   Null Set   1
     z   3


In [14]:
freqItems

[{'r'},
 {'s'},
 {'s', 'x'},
 {'y'},
 {'y', 'z'},
 {'x', 'y'},
 {'x', 'y', 'z'},
 {'t'},
 {'t', 'y'},
 {'t', 'x'},
 {'t', 'x', 'y'},
 {'t', 'z'},
 {'t', 'y', 'z'},
 {'t', 'x', 'z'},
 {'t', 'x', 'y', 'z'},
 {'x'},
 {'x', 'z'},
 {'z'}]